In [1]:
# RSI PowerZones Weekly
import gta_indicator
import gta_plot
import gta_prices
import gta_signals
import gta_statistics
import gta_trades
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Trade parameters.
exchange = 'LSE'
tidm = 'IBPO'
timeframe = 'Weekly'
filename = f'{exchange}_{tidm}_prices.csv'
history_start = '2010-09-14'
p1 = 48 # System 1 look back period.
p2 = 24 # System 2 look back period.
p3 = 12 # System 3 look back period.
p4 = 6  # System 4 look back period.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2 # Percentage risk per trade.
charges = 11.95  # Commission per trade.
stamp_duty = 0.0  # Stamp Duty percentage.

In [3]:
# Import weekly closing prices.
prices = gta_prices.weekly(exchange, tidm, history_start)

In [4]:
# Donchian channels.
dc1 = gta_indicator.donchian(prices, p1)
dc2 = gta_indicator.donchian(prices, p2)
dc3 = gta_indicator.donchian(prices, p3)
dc4 = gta_indicator.donchian(prices, p4)

In [5]:
# System 1 entries & exits.
s1 = pd.concat([prices, dc1], axis=1)
s1['buy'] = np.where(s1.close > s1.upr, 1, 0)
s1['sell'] = np.where(s1.close < s1.mid, 1, 0)
s1['state'] = gta_signals.state_signal(s1.buy, s1.sell, p1)
s1['entry'] = np.where(np.logical_and(s1.state == 1, s1.state.shift(periods=1) == 0), 1, 0)
s1['exit'] = np.where(np.logical_and(s1.state == 0, s1.state.shift(periods=1) == 1), 1, 0)

In [6]:
# System 2 entries & exits.
s2 = pd.concat([prices, dc2], axis=1)
s2['buy'] = s1.entry
s2['sell'] = np.where(s2.close < s2.mid, 1, 0)
s2['state'] = gta_signals.state_signal(s2.buy, s2.sell, p2)
s2['entry'] = np.where(np.logical_and(s2.state == 1, s2.state.shift(periods=1) == 0), 1, 0)
s2['exit'] = np.where(np.logical_and(s2.state == 0, s2.state.shift(periods=1) == 1), 1, 0)

In [7]:
# System 3 entries & exits.
s3 = pd.concat([prices, dc3], axis=1)
s3['buy'] = s1.entry
s3['sell'] = np.where(s3.close < s3.mid, 1, 0)
s3['state'] = gta_signals.state_signal(s3.buy, s3.sell, p3)
s3['entry'] = np.where(np.logical_and(s3.state == 1, s3.state.shift(periods=1) == 0), 1, 0)
s3['exit'] = np.where(np.logical_and(s3.state == 0, s3.state.shift(periods=1) == 1), 1, 0)

In [8]:
# System 4 entries & exits.
s4 = pd.concat([prices, dc4], axis=1)
s4['buy'] = s1.entry
s4['sell'] = np.where(s4.close < s4.mid, 1, 0)
s4['state'] = gta_signals.state_signal(s4.buy, s4.sell, p4)
s4['entry'] = np.where(np.logical_and(s4.state == 1, s4.state.shift(periods=1) == 0), 1, 0)
s4['exit'] = np.where(np.logical_and(s4.state == 0, s4.state.shift(periods=1) == 1), 1, 0)

In [9]:
# All entries & exits.
td = pd.concat([s1[s1.entry == 1] , s1[s1.exit == 1], s2[s2.exit == 1], s3[s3.exit == 1], s4[s4.exit == 1]], axis=0)
td = td.sort_index()

In [10]:
# Position size (buy).
td['volatility'] = np.where(td.entry == 1, abs((td.mid - td.close) / td.close).round(3), 0)
td['risk_amt'] = np.where(td.entry == 1, ((position_size * risk_pct) / td.volatility).round(2), 0)
td['shares'] = np.where(td.entry == 1, (td.risk_amt / td.close).astype('int'), 0)

# Modify number of shares to be divisible by 4.
for i in range(len(td)):
    while td.loc[td.index[i], 'shares'] % 4 != 0:
        td.loc[td.index[i], 'shares'] += 1

# Position size (sell).
for i in range(len(td)):
        if td.loc[td.index[i], 'exit'] == 1 and td.loc[td.index[i - 1], 'exit'] == 0:
            td.loc[td.index[i], 'shares'] = td.loc[td.index[i - 1], 'shares'] / 4
        elif td.loc[td.index[i], 'exit'] == 1 and td.loc[td.index[i - 1], 'exit'] == 1:
            td.loc[td.index[i], 'shares'] = td.loc[td.index[i - 1], 'shares']
td.shares = td.shares.astype('int')

In [11]:
# Costs.
td['charges'] = charges
td['stamp_duty'] = ((stamp_duty / 100) * td.close * td.shares).round(2)
td['cost'] = np.where(td.entry == 1, (td.close * td.shares) + td.charges + td.stamp_duty, (td.close * td.shares) / 4)
td.cost = td.cost.round(2)

In [12]:
# Value.
td['value'] = np.where(td.exit == 1, ((td.close * td.shares) - td.charges).round(2), 0)
td
# Fix costs calculation for selling.

,open,high,low,close,upr,lwr,mid,buy,sell,state,entry,exit,volatility,risk_amt,shares,charges,stamp_duty,cost,value
date,,,,,,,,,,,,,,,,,,,
2011-08-26,2.210,2.296,2.210,2.260,2.246,1.3500,1.79800,1,0,1,1,0,0.204,7352.94,3256,11.95,0.0,7370.51,0.00
2011-10-14,2.325,2.325,2.325,2.325,2.400,2.2600,2.33000,0,1,0,0,1,0.000,0.00,814,11.95,0.0,473.14,1880.60
2012-06-01,3.575,3.575,3.350,3.450,3.700,3.4000,3.55000,0,1,0,0,1,0.000,0.00,814,11.95,0.0,702.08,2796.35
2013-03-29,3.650,3.650,3.625,3.625,3.800,3.6000,3.70000,0,1,0,0,1,0.000,0.00,814,11.95,0.0,737.69,2938.80
2013-08-02,3.625,3.625,3.510,3.510,3.800,3.4000,3.60000,0,1,0,0,1,0.000,0.00,814,11.95,0.0,714.28,2845.19
2016-05-13,0.400,0.420,0.400,0.420,0.400,0.1075,0.25375,1,0,1,1,0,0.396,3787.88,9020,11.95,0.0,3800.35,0.00
2016-06-24,0.465,0.465,0.425,0.440,0.490,0.4000,0.44500,0,1,0,0,1,0.000,0.00,2255,11.95,0.0,248.05,980.25
2017-02-17,0.820,0.820,0.820,0.820,0.835,0.8100,0.82250,0,1,0,0,1,0.000,0.00,2255,11.95,0.0,462.28,1837.15
2017-03-10,0.820,0.820,0.775,0.775,0.835,0.7700,0.80250,0,1,0,0,1,0.000,0.00,2255,11.95,0.0,436.91,1735.68
